# Preprocesamiento de los datos - Análisis de suelos
## Importaciones

In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Carga de los datos

In [108]:
df = pd.read_csv('data/suelos_original.csv')

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Pre-procesamiento de los datos

### Renombrar columnas

In [109]:
oDict = {
    'numfila': 'id',
    'Departamento': 'departamento',
    'Municipio': 'municipio',
    'Cultivo': 'cultivo',
    'Estado': 'estado',
    'Tiempo Establecimiento': 'tiempo_establecimiento',
    'Topografia': 'topografia',
    'Drenaje': 'drenaje',
    'Riego': 'riego',
    'Fertilizantes aplicados': 'fertilizantes',
    'FechaAnalisis': 'fecha',
    'pH agua:suelo 2,5:1,0': 'ph',
    'Materia orgánica (MO) %': 'materia_organica',
    'Fósforo (P) Bray II mg/kg': 'fosforo',
    'Azufre (S) Fosfato monocalcico mg/kg': 'azufre',
    'Acidez (Al+H) KCL cmol(+)/kg': 'acidez',
    'Aluminio (Al) intercambiable cmol(+)/kg': 'aluminio',
    'Calcio (Ca) intercambiable cmol(+)/kg': 'calcio',
    'Magnesio (Mg) intercambiable cmol(+)/kg': 'magnesio',
    'Potasio (K) intercambiable cmol(+)/kg': 'potasio',
    'Sodio (Na) intercambiable cmol(+)/kg': 'sodio',
    'capacidad de intercambio cationico (CICE) suma de bases cmol(+)/kg': 'cice',
    'Conductividad el‚ctrica (CE) relacion 2,5:1,0 dS/m': 'ce',
    'Hierro (Fe) disponible olsen mg/kg': 'hierro_olsen',
    'Cobre (Cu) disponible mg/kg': 'cobre',
    'Manganeso (Mn) disponible Olsen mg/kg': 'manganeso',
    'Zinc (Zn) disponible Olsen mg/kg': 'zinc_olsen',
    'Boro (B) disponible mg/kg': 'boro',
    'Hierro (Fe) disponible doble \xa0cido mg/kg': 'hierro_doble_acido',
    'Cobre (Cu) disponible doble acido mg/kg': 'cobre_doble_acido',
    'Manganeso (Mn) disponible doble acido mg/kg': 'manganeso_doble_acido',
    'Zinc (Zn) disponible doble \xa0cido mg/kg': 'zinc_doble_acido', 
    'Secuencial': 'secuencial'
}
df.rename(columns=oDict, inplace=True)

### Eliminar columnas sin valores

In [110]:
df.drop(columns=['secuencial', 'fecha'], inplace=True)

### Formato de valores en columnas

In [111]:
# TODO capitalizar departamento y municipio
# Cambiar los valores a minúsculas
columnas = [
    'departamento', 'municipio', 'cultivo', 'estado', 'tiempo_establecimiento', 
    'topografia', 'drenaje', 'riego', 'fertilizantes'
]
for columna in columnas:
    df[columna] = df[columna].str.lower()

In [112]:
# Valores faltantes en columnas float
columnas = [
    'azufre', 'acidez', 'aluminio', 'potasio', 'sodio', 'ce','hierro_olsen', 
    'cobre', 'manganeso', 'zinc_olsen', 'boro', 'hierro_doble_acido', 
    'cobre_doble_acido', 'manganeso_doble_acido', 'zinc_doble_acido'
]
for columna in columnas:
    df[columna] = df[columna].str.replace('ND', 'NaN')

In [113]:
def clean_values(value):
    '''Limpia valores en columnas float
    
    Procesa valores string realizando las siguientes acciones:
      - Reemplaza ',' por '.'
      - Reemplaza '..' por '.'
      - Elimina el signo '>' dejando el límite superior como valor actual
      - Reemplaza los límites inferiores por la mitad de su valor
      - Reemplaza valores inválidos por NaN
    '''
    if type(value) != str:
        return value
    
    value = value.replace(',', '.')
    value = value.replace('..', '.')
    value = value.replace('>', '')
    
    if value[0] == '<':
        value = float((value[1:]).strip()) / 2        
    else:
        try:
            float(value)
        except Exception as e:
            print(str(e));
            value = 'NaN'
    return value

# Limpieza de carácteres no numéricos y conversión de valores numéricos a float
columnas = [
    'ph', 'fosforo', 'azufre', 'acidez', 'aluminio', 
    'calcio', 'magnesio', 'potasio', 'sodio', 'ce', 'hierro_olsen', 
    'cobre', 'manganeso', 'zinc_olsen', 'boro', 'hierro_doble_acido',
    'cobre_doble_acido', 'manganeso_doble_acido', 'zinc_doble_acido'
]

for columna in columnas:
    try:
        df[columna] = df[columna].apply(clean_values)
        df[columna] = df[columna].astype(float)
    except Exception as e:
        print(columna, str(e))

In [8]:
df["topografia"] = df["topografia"].str.replace("error: #n/a", "no indica")

oDict = {
    'error: #n/a': "no indica",
    'buen drenaje': 'bueno',
    'regular drenaje': 'regular',
    'mal drenaje': 'malo'
}
df["drenaje"].replace(oDict, inplace=True)

In [19]:
indices = df[
    (df["cultivo"] == "no indica") & 
    (df["estado"] == "no indica") & 
    (df["tiempo_establecimiento"] == "no indica")].index
df.drop(index=indices, axis=0, inplace=True)

## Exportar el conjunto de datos preprocesado

In [20]:
df.to_csv('data/suelos_preprocesado.csv', index=False)

In [4]:
df = pd.read_csv('data/suelos_preprocesado.csv')
df.head()

,id,departamento,municipio,cultivo,estado,tiempo_establecimiento,topografia,drenaje,riego,fertilizantes,...,ce,hierro_olsen,cobre,manganeso,zinc_olsen,boro,hierro_doble_acido,cobre_doble_acido,manganeso_doble_acido,zinc_doble_acido
0,NaN,cundinamarca,funza,uchuva,por establecer,no aplica,ondulado,bueno,no indica,no indica,...,0.223513,184.118,1.000,2.876,3.462,0.109442,NaN,NaN,NaN,NaN
1,NaN,cundinamarca,bituima,citricos,por establecer,no aplica,ondulado,bueno,no cuenta con riego,no indica,...,0.692732,33.181,3.426,1.000,90.926,0.710300,NaN,NaN,NaN,NaN
2,NaN,cundinamarca,villapinzón,pastos-brachiaria,establecido,no indica,ondulado,bueno,no cuenta con riego,no indica,...,0.334477,670.747,3.720,8.481,2.455,0.130901,NaN,NaN,NaN,NaN
3,NaN,cundinamarca,villapinzón,pastos-brachiaria,establecido,no indica,ondulado,bueno,no cuenta con riego,no indica,...,0.361426,791.175,6.830,5.694,2.631,0.302575,NaN,NaN,NaN,NaN
4,NaN,cundinamarca,"bogotá, d.c.",papa de año,por establecer,no aplica,pendiente,bueno,no cuenta con riego,"15-15-15, 18-18-18 y 10-30-10",...,0.374107,1685.377,4.124,17.619,8.567,0.302575,NaN,NaN,NaN,NaN


In [9]:
df2 = pd.read_csv('data/estaciones_preprocesado.csv')
df2.head()

,departamento,municipio,ubicacion,altitud
0,nariño,ipiales,"(0.81378611, -77.66197778)",2582
1,nariño,contadero,"(0.93030556, -77.49119444)",2450
2,nariño,túquerres,"(1.07061111, -77.63688889)",3120
3,putumayo,mocoa,"(1.08288889, -76.66711111)",760
4,amazonas,puerto nariño,"(-3.78030556, -70.36263889)",158


In [8]:
df_merged=df.merge(df2, how="inner", suffixes=(False, False))
df_merged

,id,departamento,municipio,cultivo,estado,tiempo_establecimiento,topografia,drenaje,riego,fertilizantes,...,cobre,manganeso,zinc_olsen,boro,hierro_doble_acido,cobre_doble_acido,manganeso_doble_acido,zinc_doble_acido,ubicacion,altitud
0,NaN,cundinamarca,funza,uchuva,por establecer,no aplica,ondulado,bueno,no indica,no indica,...,1.000,2.876,3.462,0.109442,NaN,NaN,NaN,NaN,"(4.73333333, -74.16666667)",2560
1,NaN,cundinamarca,funza,uchuva,por establecer,no aplica,ondulado,bueno,no indica,no indica,...,1.000,2.876,3.462,0.109442,NaN,NaN,NaN,NaN,"(4.73625, -74.15733333)",2560
2,NaN,cundinamarca,funza,uchuva,por establecer,no aplica,ondulado,bueno,no indica,no indica,...,1.000,2.876,3.462,0.109442,NaN,NaN,NaN,NaN,"(4.75905556, -74.20516667)",2560
3,NaN,cundinamarca,funza,calas,por establecer,no aplica,plano,bueno,aspersión,no indica,...,7.036,6.970,17.925,0.530354,NaN,NaN,NaN,NaN,"(4.73333333, -74.16666667)",2560
4,NaN,cundinamarca,funza,calas,por establecer,no aplica,plano,bueno,aspersión,no indica,...,7.036,6.970,17.925,0.530354,NaN,NaN,NaN,NaN,"(4.73625, -74.15733333)",2560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200180,NaN,cauca,timbiquí,plátano,establecido,de 0 a 1 año,plano,bueno,no indica,no indica,...,5.390,4.030,1.120,0.120000,NaN,NaN,NaN,NaN,"(2.87180556, -77.62166667)",30
200181,NaN,cauca,timbiquí,plátano,establecido,de 0 a 1 año,plano,bueno,no indica,no indica,...,4.860,2.360,1.000,0.100000,NaN,NaN,NaN,NaN,"(2.77361111, -77.66361111)",30
200182,NaN,cauca,timbiquí,plátano,establecido,de 0 a 1 año,plano,bueno,no indica,no indica,...,4.860,2.360,1.000,0.100000,NaN,NaN,NaN,NaN,"(2.78875, -77.46616667)",40
200183,NaN,cauca,timbiquí,plátano,establecido,de 0 a 1 año,plano,bueno,no indica,no indica,...,4.860,2.360,1.000,0.100000,NaN,NaN,NaN,NaN,"(2.87180556, -77.62166667)",30


---
Fin del notebook